In [ ]:
import requests
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
import pandas as pd
import json
import numpy as np
import dask.dataframe as dd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
import seaborn as sns


# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")

today = date.today()

# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 100)
# NumPy配列の表示オプションを設定
np.set_printoptions(threshold=np.inf)
# 配列の表示形式を科学表記ではなく通常の浮動小数点数に設定
np.set_printoptions(suppress=True)

In [ ]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [ ]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
# plt.rcParams['font.family'] = 'Meiryo'

In [ ]:
# CSVファイルを読み込む
file_path = 'data/input/user_info_cleansing.csv'  # ファイルパスを適切に設定してください
df1 = dd.read_csv(file_path).drop(['Unnamed: 0'], axis=1).compute()
file_path = 'data/input/gacha_history.csv'  # ファイルパスを適切に設定してください
df2 = dd.read_csv(file_path).compute()
file_path = 'data/input/point_history_cleansing.csv'  # ファイルパスを適切に設定してください
# column_types = {
#     'id' : np.float16,
#     'user_id' : int,
#     'series_id' : np.float16,
#     'shop_id' : np.float16,
#     'shop_name' : str,
#     'card_id' : str,
#     'リサイクル分類ID' : np.float16,
#     'amount' : np.float16,
#     'amount_kg' : np.float16,
#     'point' : np.float16,
#     'total_point' : np.float16,
#     'status' : np.float16,
#     'total_amount' : np.float16,
#     'coin' : np.float16,
#     'rank_id' : np.float16,
#     'use_date' :   'datetime64[ns]',
#     'created_at' : 'datetime64[ns]',
#     'updated_at' : 'datetime64[ns]',
#     '支店ID' : np.float16,
#     'super' : str,
#     'prefectures' : str,
#     'municipality' : str,
#     'shop_name_1' :  str,
#     'shop_id_1' :    str,
#     'created_at_1' : 'datetime64[ns]',
#     'updated_at_1' : 'datetime64[ns]',
#     'store_latitude' : np.double,
#     'store_longitude' : np.double,
# }
df3 = dd.read_csv(
    file_path,
    dtype={
        'series_id': 'Int64',
        'shop_id': 'Int64',
        'shop_id_1': str,
        'リサイクル分類ID': 'Int64',
        '支店ID': 'Int64',
        'rank_id': 'Int64'
    }
).drop(['Unnamed: 0'], axis=1).compute()

file_path = 'data/input/ユーザー基本情報_2023-12-21.csv'
df4 = dd.read_csv(file_path, encoding='sjis').compute()

データ授受日 = pd.to_datetime('2023-12-06')

# '登録日時' 列を datetime オブジェクトに変換（日付の形式は適宜調整してください）
df4['登録日時'] = pd.to_datetime(df4['登録日時'])
# サービス利用開始からの経過日数
df4['サービス利用開始からの経過日数'] = (データ授受日 - df4['登録日時']).dt.days
# サービス利用開始からの経過月数
df4['サービス利用開始からの経過月数'] = (データ授受日 - df4['登録日時']).dt.days / 30
# 経過月数が1未満は1に補正する。月数で割ると月平均が増えてしまうため
df4['サービス利用開始からの経過月数'] = df4['サービス利用開始からの経過月数'].apply(lambda x: 1 if x<1 else x)

# 不正データnan変換
df1 = df1.replace('N', np.nan)
df1 = df1.replace('nan', np.nan)
df2 = df2.replace('N', np.nan)
df2 = df2.replace('nan', np.nan)
df3 = df3.replace('N', np.nan)
df3 = df3.replace('nan', np.nan)
df4 = df4.replace('N', np.nan)
df4 = df4.replace('nan', np.nan)

#gachaのデータで100以上の獲得データはテストデータとして除外
df2 = df2[df2['add_ticket']<=100]
# 4月1日以前は削除、mission_type_idの8と9を削除
df2 = df2[(df2["mission_type_id"] != 8) & (df2["mission_type_id"] != 9) & (pd.to_datetime(df2["mission_achievement_date"]) >= pd.Timestamp('2023-04-01'))]
#pointhistoryの事務局操作データは除外
df3 = df3[df3['status'] != 3]
# 'total_amount'は全部N
df3 = df3.drop(columns=['total_amount'])

In [ ]:
# birth_dayをdatetimeに変換し、年代を計算
df1['birth_day'] = pd.to_datetime(df1['birth_day'], errors='coerce')
current_year = pd.Timestamp.now().year
df1['age'] = current_year - df1['birth_day'].dt.year
# 年齢と性別が欠損している行を削除
data_age_gender = df1.dropna(subset=['age', 'gender']).copy()
# 年齢を年代に変換
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
df1['年代'] = pd.cut(df1['age'], bins=bins, labels=labels, right=False)

# # ワンホットエンコーディング
# # gender
# df1['男'] = df1['gender'].apply(lambda x: 1 if x == '男' else 0)
# df1['女'] = df1['gender'].apply(lambda x: 1 if x == '女' else 0)
# # 年代
# df1['未成年'] = df1['gender'].apply(lambda x: 1 if x == '0-20' else 0)
# df1['20代'] = df1['gender'].apply(lambda x: 1 if x == '21-30' else 0)
# df1['30代'] = df1['gender'].apply(lambda x: 1 if x == '31-40' else 0)
# df1['40代'] = df1['gender'].apply(lambda x: 1 if x == '41-50' else 0)
# df1['50代'] = df1['gender'].apply(lambda x: 1 if x == '51-60' else 0)
# df1['60代'] = df1['gender'].apply(lambda x: 1 if x == '61-70' else 0)
# df1['70代'] = df1['gender'].apply(lambda x: 1 if x == '71-80' else 0)
# df1['80代'] = df1['gender'].apply(lambda x: 1 if x == '81-90' else 0)
# df1['90代'] = df1['gender'].apply(lambda x: 1 if x == '91-100' else 0)

結合

In [ ]:
df_user_merge = pd.merge(df1, df4, left_on='id', right_on='利用者ID', how='inner')
# 毎月平均リサイクル量
df_user_merge['毎月平均リサイクル量'] = df_user_merge['total_recycle_amount'] / df_user_merge['サービス利用開始からの経過月数']
# display(df_user_merge[['total_recycle_amount','サービス利用開始からの経過日数','毎月平均リサイクル量']])

df_merge_gacha = pd.merge(df2, df_user_merge, left_on='user_uid', right_on='id', how='left')
df_merge_gacha = df_merge_gacha.drop(['id_x','id_y'], axis=1)

df_merge_point = pd.merge(df3, df_user_merge, left_on='user_id', right_on='id', how='left')
df_merge_point = df_merge_point.drop(['id_x','id_y'], axis=1)
df_merge_point = df_merge_point[~df_merge_point['サービス利用開始からの経過日数'].isna()]
df_merge_point = df_merge_point[~df_merge_point['サービス利用開始からの経過月数'].isna()]


#### 特徴量候補を列挙
ユーザ毎
- 毎月平均リサイクル量
- 毎月平均リサイクル回数
- 毎月平均クラブコインの使用量
- 毎月平均ガチャの取得量
- 毎月平均ガチャの使用量
- 平均rank
- 店舗との距離←緯度経度をgeocodeで算出
- 店舗のリサイクル許容量
- 性別
- 年代
- カード種類
- サービス利用開始からの経過日数

### 特徴量算出

#### - 毎月平均リサイクル回数

In [ ]:
# ユーザー別リサイクル回数
count = df_merge_point.groupby('user_id').size()
count = count.to_frame('リサイクル回数')
count = count.reset_index()

df_user_merge = pd.merge(df_user_merge, count, left_on='id', right_on='user_id', how='inner')
df_user_merge['毎月平均リサイクル回数'] = df_user_merge['リサイクル回数'] / df_user_merge['サービス利用開始からの経過月数']

#### - 毎月平均クラブコインの使用量

In [ ]:
# 0以下が消費データ
used_coin = df_merge_point[df_merge_point['coin']<0].groupby('user_id')['coin'].sum()
used_coin = used_coin.to_frame('消費クラブコイン合計量').reset_index()

df_user_merge = pd.merge(df_user_merge, used_coin, left_on='id',right_on='user_id',how='left')
df_user_merge['消費クラブコイン合計量'] = df_user_merge['消費クラブコイン合計量'].fillna(0)
df_user_merge['毎月平均クラブコインの使用量'] = df_user_merge['消費クラブコイン合計量'] / df_user_merge['サービス利用開始からの経過月数']

#### - 毎月平均ガチャの取得量

In [ ]:
ガチャ取得合計 = df_merge_gacha[df_merge_gacha['add_ticket']>0].groupby('user_uid')['add_ticket'].sum()
ガチャ取得合計 = ガチャ取得合計.to_frame('ガチャ取得合計').reset_index()
# display(ガチャ取得合計)
df_user_merge = pd.merge(df_user_merge, ガチャ取得合計, left_on='id',right_on='user_uid',how='left')
df_user_merge['ガチャ取得合計'] = df_user_merge['ガチャ取得合計'].fillna(0)
df_user_merge['毎月平均ガチャの取得量'] = df_user_merge['ガチャ取得合計'] / df_user_merge['サービス利用開始からの経過月数']

#### - 毎月平均ガチャの使用量

In [ ]:
ガチャ使用量合計 = abs(df_merge_gacha[df_merge_gacha['add_ticket']<0].groupby('user_uid')['add_ticket'].sum())
ガチャ使用量合計 = ガチャ使用量合計.to_frame('ガチャ使用量合計').reset_index()
# display(ガチャ使用量合計)
df_user_merge = pd.merge(df_user_merge, ガチャ使用量合計, left_on='id',right_on='user_uid',how='left')
df_user_merge['ガチャ使用量合計'] = df_user_merge['ガチャ使用量合計'].fillna(0)
df_user_merge['毎月平均ガチャの使用量'] = df_user_merge['ガチャ使用量合計'] / df_user_merge['サービス利用開始からの経過月数']

#### - 平均rank

In [ ]:
# 'rank_id'を数値型に変換しようと試みる
# 変換できない場合はNaNを返す
df_merge_point['rank_id'] = pd.to_numeric(df_merge_point['rank_id'], errors='coerce')

平均rank = df_merge_point.groupby('user_id')['rank_id'].mean()
平均rank = 平均rank.to_frame('平均rank').reset_index()
# display(平均rank)
df_user_merge = pd.merge(df_user_merge, 平均rank, left_on='id',right_on='user_id',how='left')

#### - カード種類

In [ ]:
df_user_merge['カード種類'].value_counts()

カードの種類が「nanaco」か「みやぎ生協」かをワンホットエンコーディングする。
他は少ないのでエンコーディングしない

In [ ]:
# df_user_merge['nanaco'] = df_user_merge['カード種類'].apply(lambda x: 1 if x == 'nanaco' else 0)
# df_user_merge['みやぎ生協'] = df_user_merge['カード種類'].apply(lambda x: 1 if x == 'みやぎ生協　リサイクルポイントカード' else 0)

In [ ]:
# ユーザーIDカラム重複削除
df_user_merge = df_user_merge.drop(['user_id_x','user_id_y', 'user_uid_x', 'user_uid_y'], axis=1)
df_user_merge

#### 正規化
#### 特徴量候補を列挙
ユーザ毎
- 毎月平均リサイクル量
- 毎月平均リサイクル回数
- 毎月平均クラブコインの使用量
- 毎月平均ガチャの取得量
- 毎月平均ガチャの使用量
- 平均rank
- 店舗との距離←緯度経度をgeocodeで算出
- 店舗のリサイクル許容量
- 性別
- 年代
- カード種類
- サービス利用開始からの経過日数

In [ ]:
# # MinMaxScalerのインスタンスを作成
# scaler = MinMaxScaler()
# 正規化対象カラム = ['毎月平均リサイクル量','毎月平均リサイクル回数','毎月平均クラブコインの使用量','毎月平均ガチャの取得量','毎月平均ガチャの使用量','平均rank','サービス利用開始からの経過日数']
# # プレフィックスを付与した新しいカラム名のリストを生成
# 正規化後カラム名 = ['norm_' + col for col in 正規化対象カラム]
# # 複数カラムを指定して正規化
# df_normalized = pd.DataFrame(scaler.fit_transform(df_user_merge[正規化対象カラム]), columns=正規化後カラム名)
# # 正規化されたデータを元のデータフレームに結合
# df_user_merge = df_user_merge.join(df_normalized)

In [ ]:
# カテゴリカルデータと数値データのカラムを定義
categorical_features = [
                        'gender',
                        '年代',
                        'カード種類'
                       ]
numerical_features = [
                      '毎月平均リサイクル量',
                      '毎月平均リサイクル回数',
                      '毎月平均クラブコインの使用量',
                      '毎月平均ガチャの取得量',
                      '毎月平均ガチャの使用量',
                      '平均rank',
                      'サービス利用開始からの経過日数'
                     ]

# 前処理パイプライン
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)])

# データを前処理
X_processed = preprocessor.fit_transform(df_user_merge)

# K-meansモデルの初期化
kmeans = KMeans(n_clusters=5, n_init=10, random_state=None) #n_init='auto'となっていましたが、chef手元でエラー出たためn_init=10に変更

# モデルの訓練
kmeans.fit(X_processed)

# クラスタリング結果の取得
df_user_merge['ラベル'] = kmeans.labels_

display(df_user_merge['ラベル'].value_counts())
# 各クラスタの統計を計算
# 数値型のデータのみを含む列を選択
numeric_cols = df_user_merge.select_dtypes(include=['number'])

# クラスタラベルを追加
numeric_cols['ラベル'] = kmeans.labels_

cluster_stats = numeric_cols.groupby('ラベル').agg(['mean', 'median', 'std'])

print(cluster_stats)

### 分析の方法
クラスタ内の統計:

各クラスタの要約統計（平均、中央値、標準偏差など）を計算して、クラスタの特性を理解します。
特徴量の重要性:

クラスタごとの特徴量の平均値や分布を分析し、どの特徴量がクラスタ形成に最も影響を与えているかを評価します。
クラスタの比較:

異なるクラスタを比較して、それらがどのように異なるか、または共通する特性を持っているかを分析します。
ドメイン知識の適用:

ドメイン知識を適用して、クラスタの結果をビジネスや研究の文脈で解釈します。

### 可視化の方法
散布図:

2次元または3次元の散布図を使用して、クラスタリングされたデータポイントをプロットします。各クラスタは異なる色やマーカーで表示します。
高次元データの場合、主成分分析（PCA）やt-SNEなどの次元削減技術を使用して、データを2Dまたは3D空間にマッピングできます。
ペアプロット:

各特徴量ペアの組み合わせに基づいて散布図のグリッドを作成します。これにより、特徴量間の関係とクラスタの分布を詳細に調べることができます。
ヒートマップ:

クラスタの中心点をヒートマップで表示し、各クラスタの特徴を比較します。
シルエットプロット:

クラスタリングの品質を評価するためにシルエットスコアを可視化します。このスコアは、クラスタ内の密度とクラスタ間の分離を測定します。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
# 2D PCA でのデータのプロット
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_processed)
plt.scatter(X_pca[:, 0], X_pca[:, 1],alpha=0.3,s=20, c=kmeans.labels_)
plt.title("PCA-based Scatter Plot of Clusters")
plt.xlim(0,20)
# plt.ylim(-10,10)
plt.show()

# シルエットスコアの計算とプロット
silhouette_vals = silhouette_samples(X_processed, kmeans.labels_)
# シルエットプロットの実装コード...


### KMeansの結果可視化

##### KMeansのクラスタリング結果について、任意の説明変数2つで散布図を描写するなどして、直感的な理解を得る。

In [ ]:
# 説明変数のペアを選択
x = '毎月平均リサイクル量'
y = '平均rank'
# 散布図の作成
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_user_merge, x=x, y=y, hue='ラベル', palette='bright', s=30, alpha=0.5)
plt.title('クラスタリング結果の散布図')
plt.xlabel(x)
plt.ylabel(y)
plt.legend(title='ラベル', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlim(0, 500)
#plt.ylim(0,100)
plt.show()

In [ ]:
# 説明変数のペアを選択
x = '毎月平均リサイクル量'
y = 'サービス利用開始からの経過日数'
# 散布図の作成
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_user_merge, x=x, y=y, hue='ラベル', palette='bright', s=30, alpha=0.5)
plt.title('クラスタリング結果の散布図')
plt.xlabel(x)
plt.ylabel(y)
plt.legend(title='ラベル', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlim(0, 500)
#plt.ylim(0,100)
plt.show()

In [ ]:
# 説明変数のペアを選択
x = '毎月平均クラブコインの使用量'
y = 'サービス利用開始からの経過日数'
# 散布図の作成
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_user_merge, x=x, y=y, hue='ラベル', palette='bright', s=30, alpha=0.5)
plt.title('クラスタリング結果の散布図')
plt.xlabel(x)
plt.ylabel(y)
plt.legend(title='ラベル', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlim(-2000,0)
#plt.ylim(0,100)
plt.show()
# この日しかクラブコインを使えない見たいなルールがあるのか？

In [ ]:
# 説明変数のペアを選択
x = '毎月平均ガチャの取得量'
y = '毎月平均ガチャの使用量'
# 散布図の作成
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_user_merge, x=x, y=y, hue='ラベル', palette='bright', s=30, alpha=0.5)
plt.title('クラスタリング結果の散布図')
plt.xlabel(x)
plt.ylabel(y)
plt.legend(title='ラベル', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.xlim(-2000,0)
#plt.ylim(0,100)
plt.show()

# 特定の説明変数を選択
variable = '毎月平均ガチャの取得量'
# 箱ひげ図
plt.figure(figsize=(10, 6))
sns.boxplot(x='ラベル', y=variable, data=df_user_merge)
plt.title(f'{variable} のクラスタリング結果（箱ひげ図）')
plt.show()

# ラベルごとのデータ数
label_counts = df_user_merge.groupby('ラベル').size()
formatted_counts = ', '.join([f"{label}:{count}" for label, count in label_counts.items()])
print("ラベルごとのデータ数:", formatted_counts)

In [ ]:
# 説明変数のペアを選択
x = '平均rank'
y = 'サービス利用開始からの経過日数'
# 散布図の作成
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_user_merge, x=x, y=y, hue='ラベル', palette='bright', s=50, alpha=0.5)
plt.title('クラスタリング結果の散布図')
plt.xlabel(x)
plt.ylabel(y)
plt.legend(title='ラベル', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.xlim(0,1000)
#plt.ylim(0,100)
plt.show()

#### 以下の5グループに分かれる
経過日数が浅く、あまりリサイクルしていないグループ<br>
経過日数が浅く、ログインしてガチャをたくさん回しているグループ<br>
経過日数が長く、あまりリサイクルしていないグループ<br>
リサイクルガチ勢<br>
ガチャガチ勢<br>

In [ ]:
#性別
# 説明変数のペアを選択
x = 'gender'
y = '毎月平均リサイクル量'
# 散布図の作成
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_user_merge, x=x, y=y, hue='ラベル', palette='bright', s=80, alpha=0.7)
plt.title('クラスタリング結果の散布図')
plt.xlabel(x)
plt.ylabel(y)
plt.legend(title='ラベル', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
#カード種類
# 説明変数のペアを選択
x = 'カード種類'
y = '毎月平均リサイクル量'
# 散布図の作成
plt.figure(figsize=(20, 12))
sns.scatterplot(data=df_user_merge, x=x, y=y, hue='ラベル', palette='bright', s=80, alpha=0.7)
plt.title('クラスタリング結果の散布図')
plt.xlabel(x)
plt.ylabel(y)
plt.legend(title='ラベル', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

店舗との距離←緯度経度をgeocodeで算出<br>

レーダーチャート
平行座標プロット
ツリーマップ（Tree Maps）またはサンキーダイアグラム（Sankey Diagrams）<br>
→ツリーマップは主に単一の変数（例えばユーザー数や収益など）の分布を示すために使用されます。各四角形の面積はその変数の大きさを表し、階層的なデータ構造を視覚化するのに適していますが、複数の変数を同時に表現することには限界があるため今回不採用<br>
サンキーダイアグラムは時間の経過に伴うクラスタの変化、ユーザーの行動パターンなどを表すときに使う状態変化量を可視化するものなので今回不採用

In [ ]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
use_columns = ['毎月平均リサイクル量', '平均rank', 'サービス利用開始からの経過日数','毎月平均ガチャの取得量','age']
df_user_merge[use_columns] = scaler.fit_transform(df_user_merge[use_columns])

#正規化
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# use_columns = ['毎月平均リサイクル量', '平均rank', 'サービス利用開始からの経過日数', '毎月平均ガチャの取得量','age']
# df_user_merge[use_columns] = scaler.fit_transform(df_user_merge[use_columns])


# クラスタごとの平均値計算
cluster_avgs = df_user_merge[use_columns + ['ラベル']].groupby('ラベル').mean()

# レーダーチャートの描画
labels=np.array(use_columns)
num_vars = len(labels)

angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

for idx, row in cluster_avgs.iterrows():
    values = row.values.tolist()
    display(values)
    values += values[:1]
    ax.plot(angles, values, label='Cluster {}'.format(idx))

ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)
ax.set_thetagrids(np.degrees(angles[:-1]), labels)

plt.legend()
plt.show()
# TODO: リサイクル量正規化しておかしくなってないか？

In [ ]:
# 平行座標プロット
from pandas.plotting import parallel_coordinates
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
parallel_coordinates(df_user_merge[use_columns + ['ラベル']], 'ラベル', color=['#FF5733', '#33FFCE', '#335BFF'], alpha=0.3)
plt.show()


TODO: クラスタリング結果はcsv化しよう<br>
データ分類の教師あり学習のロジスティック回帰(継続利用するかしないか)、サポートベクターマシンも試したい。

In [ ]:

df_user_merge.groupby('ラベル')['gender'].value_counts()